In [1]:
import os
from os import listdir
from os.path import isfile, join

import numpy as np
import torch
from torchvision import models

TMP_PT = 'tmp.pt'

# to execute locally change path, not in repo
IMAGENET_VAL_ROOT = '../../tmp/imgnet/ILSVRC2012_img_val'

In [2]:

image_sizes = [os.path.getsize(join(IMAGENET_VAL_ROOT, f)) for f in listdir(IMAGENET_VAL_ROOT) if
               isfile(join(IMAGENET_VAL_ROOT, f))]
image_sizes = np.array(image_sizes)

In [3]:
print('Average image size in bytes: ')
avg_image_size = np.average(image_sizes)
print(avg_image_size)
print('Median image size in bytes: ')
print(np.median(image_sizes))

print('Average image size in kilobytes: ')
print(np.average(image_sizes) / 1000)
print('Median image size in kilobytes: ')
print(np.median(image_sizes) / 1000)

Average image size in bytes: 
134131.20638
Median image size in bytes: 
126689.0
Average image size in kilobytes: 
134.13120637999998
Median image size in kilobytes: 
126.689


In [4]:
models = [models.mobilenet_v2, models.googlenet, models.resnet18, models.resnet50, models.resnet152]
model_names = [m.__name__ for m in models]

In [5]:
state_dict_sizes = []

for model in models:
    tmp_model = model(pretrained=True)
    torch.save(tmp_model.state_dict(), TMP_PT)
    state_dict_size = os.path.getsize(TMP_PT)
    state_dict_sizes.append(state_dict_size)
    print('{} \t state_dict size: \t {} megabyte'.format(model.__name__, state_dict_size/ 10 ** 6))
    os.remove(TMP_PT)


mobilenet_v2 	 state_dict size: 	 14.266675 megabyte
googlenet 	 state_dict size: 	 26.681183 megabyte
resnet18 	 state_dict size: 	 46.837875 megabyte
resnet50 	 state_dict size: 	 102.548719 megabyte
resnet152 	 state_dict size: 	 241.692183 megabyte


In [6]:
print("Uncompressed images")
for model_name, dict_size in zip(model_names, state_dict_sizes):
    per_state_dict = dict_size / avg_image_size
    print('{} \t images instead of state_dict: \t {}'.format(model_name, per_state_dict))

Uncompressed images
mobilenet_v2 	 images instead of state_dict: 	 106.36357776117991
googlenet 	 images instead of state_dict: 	 198.91853447147085
resnet18 	 images instead of state_dict: 	 349.1944661058673
resnet50 	 images instead of state_dict: 	 764.5403464833879
resnet152 	 images instead of state_dict: 	 1801.9086648283378
